In [ ]:
#Silence Trimming
!pip install pydub
import os
from pydub import AudioSegment
import librosa
import numpy as np
import soundfile as sf

# Silence trimming function
def trim_silence(audio_path, top_db=20):
    # Load audio file (wav)
    y, sr = librosa.load(audio_path)

    # Trim the beginning and ending silence below a threshold
    yt, index = librosa.effects.trim(y, top_db=top_db)

    return yt, sr

# Process function for each folder and save in new directories
def process_and_save(base_dir, new_base_dir):
    if not os.path.exists(new_base_dir):
        os.makedirs(new_base_dir)

    for filename in os.listdir(base_dir):
        if filename.endswith(".wav"):
            old_file_path = os.path.join(base_dir, filename)
            new_file_path = os.path.join(new_base_dir, filename)

            audio_data, sr = trim_silence(old_file_path)
            sf.write(new_file_path, audio_data, sr)

# Define directories and folders
human_dirs = '/content/drive/MyDrive/음성데이터셋/03fin48_human_koen_padding_cut_no_noise'
new_human_dirs = '/content/drive/MyDrive/음성데이터셋/03fin48_human_koen_padding_cut_no_noise'

# Process human dirs
process_and_save(human_dirs, new_human_dirs)

In [ ]:
#Silence Trimming
!pip install pydub
import os
from pydub import AudioSegment
import librosa
import numpy as np
import soundfile as sf

# Silence trimming function
def trim_silence(audio_path, top_db=20):
    # Load audio file (wav)
    y, sr = librosa.load(audio_path)

    # Trim the beginning and ending silence below a threshold
    yt, index = librosa.effects.trim(y, top_db=top_db)

    return yt, sr

# Process function for each folder and save in new directories
def process_and_save(base_dir, new_base_dir):
    if not os.path.exists(new_base_dir):
        os.makedirs(new_base_dir)

    for filename in os.listdir(base_dir):
        if filename.endswith(".wav"):
            old_file_path = os.path.join(base_dir, filename)
            new_file_path = os.path.join(new_base_dir, filename)

            audio_data, sr = trim_silence(old_file_path)
            sf.write(new_file_path, audio_data, sr)

# Define directories and folders
human_dirs = '/content/drive/MyDrive/음성데이터셋/03fin48_ai_koen_padding_cut_no_noise'
new_human_dirs = '/content/drive/MyDrive/음성데이터셋/03fin48_ai_koen_padding_cut_no_noise'

# Process human dirs
process_and_save(human_dirs, new_human_dirs)

In [ ]:
#rnnoise이용해서 노이즈 제거하기
!git clone https://gitlab.xiph.org/xiph/rnnoise.git
!pip install soundfile pydub
!apt install -y ffmpeg
!sudo apt-get install -y autoconf automake libtool gcc

%cd rnnoise
!./autogen.sh
!./configure
!make

import os

# 현재 디렉토리의 .libs 폴더에 librnnoise.so.0가 있다고 가정하였습니다.
# 실제 경로에 맞게 수정해야 합니다.
lib_path = os.path.join(os.getcwd(), '.libs')

if lib_path not in os.environ['LD_LIBRARY_PATH']:
    os.environ['LD_LIBRARY_PATH'] += ':' + lib_path

from pydub import AudioSegment
import soundfile as sf
import subprocess


# MP3 to WAV 변환하기
subprocess.run(['ffmpeg', '-i', '/content/drive/MyDrive/PBL/ai_voice/ainoise.mp3', 'input.wav'])

# WAV to RAW PCM 변환하기
data, samplerate = sf.read('input.wav')
sf.write('input.raw', data, samplerate, subtype='PCM_16')

# rnnoise_demo 실행 파일 호출하기
subprocess.run(['./examples/.libs/rnnoise_demo', 'input.raw', 'output.raw'])

# RAW PCM to WAV 변환하기
data, samplerate = sf.read('output.raw', channels=1, samplerate=48000, subtype='PCM_16')
sf.write('/content/drive/MyDrive/PBL/ai_voice/denoised.wav', data, samplerate)

print("Denoising completed. The denoised audio is saved as '/content/drive/MyDrive/PBL/ai_voice/denoised.wav'.")